In [141]:
import requests
import numpy as np

In [7]:
#### standard post requests######
# defining the api-endpoint  
API_ENDPOINT = "http://pastebin.com/api/api_post.php"
  
# your API key here 
API_KEY = "XXXXXXXXXXXXXXXXX"
  
# your source code here 
source_code = ''' 
print("Hello, world!") 
a = 1 
b = 2 
print(a + b) 
'''
  
# data to be sent to api 
data = {'api_dev_key':API_KEY, 
        'api_option':'paste', 
        'api_paste_code':source_code, 
        'api_paste_format':'python'} 
  
# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data) 
  
# extracting response text  
pastebin_url = r.text 
print("The pastebin URL is:%s"%pastebin_url) 

The pastebin URL is:Bad API request, invalid api_dev_key


In [9]:
####### standard get requst#####
# api-endpoint 
URL = "http://maps.googleapis.com/maps/api/geocode/json"
  
# location given here 
location = "delhi technological university"
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 
  
  
print(data)

{'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account', 'results': [], 'status': 'REQUEST_DENIED'}


In [100]:
import json
import re


request_header_string = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''



def parse_string_to_dict(request_header_string):
    request_header_dict={}
    request_header_list = [header.split(": ") if len(header.split(": ")) == 2 else [header.split(": ")[0],header.split(': ')[1]+': '+header.split(': ')[2]] for header in request_header_string.split('\n')]
#     print (request_header_list)
    for (dict_key,dict_value) in request_header_list:
        request_header_dict[dict_key] = dict_value
    return request_header_dict


In [102]:
URL = "http://192.168.0.10/read_file.php?filename=defaultParameters.txt&time=0"
  
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = parse_string_to_dict(request_header_string)
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 
  
response.text

'"1350\\n2450\\n1\\n16044\\n10\\n1100\\n51\\n1726\\n" \n\n'

In [ ]:
"1350\n2450\n1\n16044\n10\n1100\n51\n1726\n" 

# Calibration script

In [151]:
### read file script 1
header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''


PARAMS = parse_string_to_dict(header)
URL = "http://192.168.0.10/read_file.php?filename=calibration_data%2Fpixel_coeffs.txt&time=0"
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 
  
response.text


'"-1650.2637939453125000000000000000000000000000000000000000000000000000\\n1.1157624721527099609375000000000000000000000000000000000000000000\\n0.0001071036967914551496505737304687500000000000000000000000000000\\n" \n\n'

In [152]:
#### write file script 1
n_random= np.random.randint(10,300)

write_file_body={"a":''' exec 1> log.txt
rm patterns/*.csv
rm patterns/*.bmp
rm patterns/*.sdf
rm tmp/*
rm *.bmp
echo "Generating Patterns..." > tmp/script_status{n}.txt
dlp_nirscan -A51 -Z1726 -N110
if [ $? -eq 255 ];
 then
 echo "failed" > tmp/script_status{n}.txt
 exit 0
 fi
cd patterns/
echo "Preparing Scan Solution..." > ../tmp/script_status10.txt
dlp_nirscan -Pscan.sdf
if [ $? -eq 255 ];
 then
 echo "failed" > ../tmp/script_status{n}.txt
 exit 0
 fi
echo "Loading Patterns..." > ../tmp/script_status{n}.txt
count=0
for entry in *
do
case $entry in *.bmp)
convert $entry -virtual-pixel Black  -filter point -interpolate nearestneighbor -distort polynomial "2 $(cat /usr/share/matrix-gui-2.0/calibration_data/control_points.txt)" scan_img.bmp
mv scan_img.bmp $entry
count=$((count+1));;
esac
done
dlp_nirscan -l$count

cd .. 
dlp_nirscan -S110 -E1400 -fcustom_scan
#Using a more unique string to detect if the script is completed
echo "_?!!SCRIPT_COMPLETED!!?_" > tmp/script_status{n}.txt'''.format(n=n_random),
'filename': 'scan.sh'}



header_string = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Content-Length: 1185
Content-Type: application/x-www-form-urlencoded
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Origin: http://192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''

header = parse_string_to_dict(header_string)

data = write_file_body
API_ENDPOINT = "http://192.168.0.10/write_file.php"
# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data, headers=header) 
  
# extracting response text  
print(r.text)

In [153]:
### read file script 1
header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''


PARAMS = parse_string_to_dict(header)
URL = "http://192.168.0.10/run_scan.php"
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 
  
response.text


''

In [154]:
#### write file script 2
body = {'a': '''1350
2450
10
1610
10
110
51
1726''',
'filename': 'defaultParameters.txt'}



header_string = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Content-Length: 82
Content-Type: application/x-www-form-urlencoded
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Origin: http://192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''

header = parse_string_to_dict(header_string)
data = body

API_ENDPOINT = "http://192.168.0.10/write_file.php"
# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data, headers=header) 
  
# extracting response text  
print(r.text)

In [155]:
### check ccalibration status###
### read file script 1
import time
header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''


PARAMS = parse_string_to_dict(header)
URL = "http://192.168.0.10/tmp/script_status{n}.txt".format(n=n_random)
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 

while response.status_code == 404:
    print(response.text)
    time.sleep(2)
    response = requests.get(url = URL, params = PARAMS)
response.text


'_?!!SCRIPT_COMPLETED!!?_\n'

In [156]:
n_random

90